In [44]:
##(Question 8)

import nltk, re,pprint

grammar = nltk.CFG.fromstring("""S -> NP VP 
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

e_chart_parser2 = nltk.EarleyChartParser(grammar, trace=2)
trees = e_chart_parser2.parse(sent)

|.  I  . shot.  an .eleph.  in .  my .pajam.|
Leaf Init Rule:
|[-----]     .     .     .     .     .     .| [0:1] 'I'
|.     [-----]     .     .     .     .     .| [1:2] 'shot'
|.     .     [-----]     .     .     .     .| [2:3] 'an'
|.     .     .     [-----]     .     .     .| [3:4] 'elephant'
|.     .     .     .     [-----]     .     .| [4:5] 'in'
|.     .     .     .     .     [-----]     .| [5:6] 'my'
|.     .     .     .     .     .     [-----]| [6:7] 'pajamas'
Top Down Init Rule:
|>     .     .     .     .     .     .     .| [0:0] S  -> * NP VP

* Processing queue: 0 

Predictor Rule:
|>     .     .     .     .     .     .     .| [0:0] NP -> * Det N
|>     .     .     .     .     .     .     .| [0:0] NP -> * Det N PP
|>     .     .     .     .     .     .     .| [0:0] NP -> * 'I'

* Processing queue: 1 

Scanner Rule:
|[-----]     .     .     .     .     .     .| [0:1] NP -> 'I' *
Completer Rule:
|[----->     .     .     .     .     .     .| [0:1] S  -> NP * VP
Predictor Rule:


In [59]:
## (Question 9)

tokens = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

grammar = nltk.CFG.fromstring("""S -> NP VP 
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    # fill w/ dots
    wfst = [["." for i in range(numtokens+1)]
        for j in range(numtokens+1)]

    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst 

def complete_wfst(wfst, tokens, trace=False):
    index = {}
    for prod in grammar.productions(): #make reverse lookup
        index[prod.rhs()] = prod.lhs()
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span): #go down diagonal
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if (nt1,nt2) in index:
                    if trace:
                        print ("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" %\
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
                    wfst[start][end] = index[(nt1,nt2)]
    return wfst 

def display(wfst, tokens):
    print ('\nWFST' + ' '.join([("%-4d" % i) for i in range(1, len(wfst))]))
    for i in range(len(wfst)-1):
        print ("%d " % i,)
        for j in range(1, len(wfst)):
            print ("%-4s" % wfst[i][j],)
        print 
      
tokens = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
#wfst0 = init_wfst(tokens, grammar)
#display(wfst0, tokens)
#wfst1 = complete_wfst(wfst0,tokens)
#display(wfst1, tokens)
wfst2 =complete_wfst(wfst0,tokens,trace=True)



[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]


In [62]:
##(Question 11)

import nltk

nltk.download('universal_tagset')
bw = nltk.corpus.brown.words()
bwt = nltk.corpus.brown.tagged_words()
bwts = nltk.corpus.brown.tagged_words(tagset='universal')



##Write programs to process the Brown Corpus and find answers to the following questions:
#        Which nouns are more common in their plural form, rather than
#        their singular form? (Only consider regular plurals, formed
#        with the -s suffix.)
freq = nltk.defaultdict(int)
for w in [w.lower() for (w,t) in bwt if t.startswith('N')]:
    freq[w] +=1
print ("\nWhich Nouns are more common as plural?\n")
for w in freq:
    if w+'s' in freq and freq[w+'s'] > freq[w]:
        print ("%s (%d, %d)" % (w, freq[w+'s'], freq[w]))


#        Which word has the greatest number of distinct tags. What are they, and what do they represent?

print ("\nWord with the most distinct tags\n")
word_tag = nltk.defaultdict(set)
for w,t  in  bwt:
    word_tag[w.lower()].add(t)
m = max(len(word_tag[w]) for w in word_tag)
print  ([(w, t) for w,t  in word_tag.items() if len(t) == m])

#        List tags in order of decreasing frequency. What do the 20 most frequent tags represent?
print ("\nTop 20 tags in decreasing frequency\n")

tagfd = nltk.FreqDist(t for (w,t)  in  bwt)
print (tagfd.most_common(20))

print ("\nTop 20 simplified tags in decreasing frequency\n")

tagfd = nltk.FreqDist(t for (w,t)  in  bwts)
print (tagfd.most_common(20))

#        Which tags are nouns most commonly found after? What do these tags represent? 

print ("\nTop 20 simplified tags that proceed nouns\n")
tagbigram =nltk.bigrams(t for (w,t)  in  bwts)

afterN = nltk.FreqDist(t1 for (t1,t2)  in  tagbigram if t2 == 'NOUN')

print (afterN.most_common(20))

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\My\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!



Which Nouns are more common as plural?

term (160, 76)
year (950, 658)
candidate (38, 34)
official (62, 20)
bond (47, 46)
item (72, 55)
fund (95, 62)
legislator (20, 7)
event (101, 81)
keynote (2, 1)
revision (9, 8)
requirement (83, 27)
fee (29, 16)
appointee (5, 2)
poll (10, 9)
texan (9, 4)
disclosure (4, 2)
minute (196, 44)
worker (86, 30)
employer (17, 15)
student (213, 131)
dollar (97, 46)
appropriation (9, 5)
leader (107, 74)
thing (368, 333)
rule (82, 66)
hour (174, 145)
element (107, 52)
step (118, 110)
diplomat (6, 5)
affair (84, 33)
lao (64, 17)
observer (20, 16)
complication (5, 4)
factor (105, 71)
nation (175, 139)
vehicle (53, 35)
eye (401, 122)
expert (38, 19)
voter (20, 4)
member (325, 137)
product (108, 87)
remark (46, 29)
passenger (21, 14)
lip (69, 18)
freeholder (3, 1)
representative (41, 29)
wave (51, 44)
visitor (36, 13)
scientist (36, 17)
object (62, 53)
locale (4, 3)
norm (24, 10)
democrat (40, 13)
month (189, 130)
owner (35, 33)
american (93, 34)
banker (15, 5)


tear (34, 2)
probe (3, 1)
supporter (8, 3)
evocation (2, 1)
barricade (4, 2)
bug (7, 4)
tidbit (2, 1)
bidder (3, 2)
libertie (8, 1)
interrelationship (2, 1)
ion (9, 6)
apologie (4, 1)
conformist (3, 2)
reinforcement (3, 2)
subordinate (9, 1)
disadvantage (9, 4)
hulk (3, 2)
survivalist (2, 1)
libertine (2, 1)
allusion (5, 3)
lodging (2, 1)
generalization (7, 4)
exploit (4, 1)
bootlegger (2, 1)
epithet (7, 2)
robot (3, 1)
dystopia (8, 2)
ad (10, 5)
brave (3, 1)
improvisation (2, 1)
aberration (5, 3)
european (5, 2)
bruise (8, 2)
cooperative (10, 1)
as (5, 1)
headquarter (65, 1)
assessor (20, 2)
authorization (6, 2)
manual (4, 2)
hydride (3, 2)
resistor (7, 3)
terminal (4, 3)
infestation (4, 2)
rupee (14, 6)
receipt (7, 4)
proceeding (18, 5)
alien (3, 2)
dependent (2, 1)
antecedent (2, 1)
polymer (4, 1)
deadlines (2, 1)
centimeter (8, 2)
lobe (5, 3)
pore (3, 2)
ripple (5, 4)
shunt (5, 1)
parameter (8, 7)
ellipsoid (4, 1)
active (8, 6)
surfactant (2, 1)
polyphosphate (5, 1)
reagent (3, 1)
